In [ ]:
import os
import hashlib
import numpy as np
import pandas as pd
import librosa
import librosa.display
import noisereduce as nr



#AUDIO_DIR = r"" define ruta donde guardas tus audios

TARGET_SR = 16000                        # Frecuencia de muestreo deseada (Hz) samples por segundo, para capturar         información vocal, formato ideal ya que es liviano
DESIRED_LENGTH = 5 * TARGET_SR           # Duración fija en samples (5 segundos = 5*16000 = 80000) padding para modelos de ML



def file_hash(filepath):
    """
    Calcula un hash MD5 único para un archivo de audio.
    - Sirve para identificar duplicados exactos en el dataset.
    """
    with open(filepath, "rb") as f:
        return hashlib.md5(f.read()).hexdigest()


def preprocess_audio(filepath, target_sr=TARGET_SR, desired_length=DESIRED_LENGTH):
    """
    Preprocesa un archivo de audio siguiendo un pipeline:
    1. Carga
    2. Trimming (quita silencios iniciales y finales)
    3. Denoise (reduce ruido de fondo)
    4. Normalización de amplitud
    5. Resampling (ajusta la frecuencia de muestreo a target_sr)
    6. Padding/Truncating (ajusta longitud a desired_length)

    Devuelve:
    - x: señal de audio preprocesada (numpy array)
    - sr: frecuencia de muestreo final
    """
    try:
        # Cargar audio con su frecuencia original
        x, sr = librosa.load(filepath, sr=None)

        # 1. Trimming → eliminar silencios, se utiliza un estandar de 35 db para eliminar silencios
        x, _ = librosa.effects.trim(x, top_db=35)

        # 2. Denoising → reducir ruido, por ejemplo sonidos de ambiente 
        x = nr.reduce_noise(y=x, sr=sr)

        # 3. Normalización → amplitud uniforme
        x = librosa.util.normalize(x)

        # 4. Resampling → a TARGET_SR, se usa el target_Sr para estandarizar la frecuencia de muestreo, samples por segundo
        if sr != target_sr:
            x = librosa.resample(x, orig_sr=sr, target_sr=target_sr)

        # 5. Padding / Truncating → duración fija de los audios
        if len(x) < desired_length:
            x = np.pad(x, (0, desired_length - len(x)))
        else:
            x = x[:desired_length]

        return x, target_sr

    except Exception as e:
        print(f"Error procesando {filepath}: {e}")
        return None, None


def extract_features(x, sr):
    """
    Extrae características principales (features) de un audio preprocesado:
    - Zero Crossing Rate (zcr): Cuántas veces la onda del audio cruza el cero (cambia de positivo a negativo).
    - RMS Energy (rms): La energía promedio del audio
    - Spectral Centroid (centroid): El “centro de gravedad” del espectro de frecuencias. Indica si el sonido tiende a ser más grave o más agudo.
    - Spectral Rolloff (rolloff): El punto de frecuencia donde está acumulado el 85–95% de la energía del sonido. Relacionado con qué tan brilloso o apagado suena.

    - MFCCs (mfcc1–mfcc13): Representa la energía global del sonido (similar a un promedio de todo el espectro).
    """
    features = {}

    # Zero Crossing Rate
    features["zcr"] = np.mean(librosa.feature.zero_crossing_rate(y=x))

    # RMS Energy
    features["rms"] = np.mean(librosa.feature.rms(y=x))

    # Spectral Centroid
    features["centroid"] = np.mean(librosa.feature.spectral_centroid(y=x, sr=sr))

    # Spectral Rolloff
    features["rolloff"] = np.mean(librosa.feature.spectral_rolloff(y=x, sr=sr))

    # MFCCs (13 coeficientes)
    mfccs = librosa.feature.mfcc(y=x, sr=sr, n_mfcc=13)
    for i in range(1, 14):
        features[f"mfcc{i}"] = np.mean(mfccs[i-1])

    return features


def build_dataset(audio_dir=AUDIO_DIR):
    """
    Construye el dataset de features a partir de los audios en una carpeta.

    Pasos:
    1. Recorrer todos los archivos de audio
    2. Eliminar archivos vacíos y duplicados
    3. Preprocesar audio
    4. Extraer features
    5. Guardar en DataFrame limpio (sin duplicados ni nulos)

    Devuelve:
    - DataFrame con los features de todos los audios
    """
    data = []
    seen_hashes = set()   # Para detectar duplicados

    for root, _, files in os.walk(audio_dir):
        for fname in files:
            if fname.endswith((".wav", ".mp3", ".flac")):
                fpath = os.path.join(root, fname)

                # 1. Limpieza dataset crudo → eliminar audios vacíos
                if os.path.getsize(fpath) == 0:
                    print(f"⏭️ Saltando archivo vacío: {fpath}")
                    continue

                # 2. Eliminar duplicados exactos usando hash
                h = file_hash(fpath)
                if h in seen_hashes:
                    print(f"⏭️ Saltando duplicado: {fpath}")
                    continue
                seen_hashes.add(h)

                # 3. Preprocesamiento
                x, sr = preprocess_audio(fpath)
                if x is None:
                    continue

                # 4. Extracción de features
                feats = extract_features(x, sr)
                feats["filename"] = fname  
                data.append(feats)

    # 5. Crear DataFrame y limpieza tabular
    df = pd.DataFrame(data)
    df = df.drop_duplicates()   # eliminar duplicados tabulares
    df = df.dropna()            # eliminar filas con nulos

    return df



df_features = build_dataset(AUDIO_DIR)

# Guardar resultados en CSV
df_features.to_csv("audio_features.csv", index=False)

print("Dataset procesado y guardado en audio_features.csv")
print(df_features.head())


Error convirtiendo C:\Users\jlpar\Escritorio\preprocesamiento audios\Mineria de datos\clipsprueba\common_voice_es_40198068.mp3: [WinError 2] El sistema no puede encontrar el archivo especificado
✅ Features guardados en features_final.csv
Empty DataFrame
Columns: []
Index: []


c:\Users\jlpar\Escritorio\preprocesamiento audios\preprocesamiento\Lib\site-packages\pydub\utils.py:170: RuntimeWarning: Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work
  warn("Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work", RuntimeWarning)
c:\Users\jlpar\Escritorio\preprocesamiento audios\preprocesamiento\Lib\site-packages\pydub\utils.py:198: RuntimeWarning: Couldn't find ffprobe or avprobe - defaulting to ffprobe, but may not work
  warn("Couldn't find ffprobe or avprobe - defaulting to ffprobe, but may not work", RuntimeWarning)
